# 3.4 Data Sharing
Science is much more impactful once it's shared. Therefore, we are going to learn how to 
open up our scientific output from a cloud platform, so that is openly available - and 
has the chance to make the impact it should.
- Load data
- ...


In [1]:
import openeo
from sentinelhub import (SHConfig, SentinelHubRequest, DataCollection, MimeType, CRS, BBox, bbox_to_dimensions, geometry)

import eo_utils

In [2]:
config = SHConfig()
config.sh_client_id = %env SH_CLIENT_ID
config.sh_client_secret = %env SH_CLIENT_SECRET

In [4]:
conn = openeo.connect('https://jjdxlu8vwl.execute-api.eu-central-1.amazonaws.com/production')

In [5]:
conn.authenticate_basic(username=config.sh_client_id, password=config.sh_client_secret)

<Connection to 'https://jjdxlu8vwl.execute-api.eu-central-1.amazonaws.com/production/' with BasicBearerAuth>

In [ ]:
# Use this for more 
# https://github.com/openEOPlatform/sample-notebooks/blob/main/openEO%20Platfrom%20-%20Basics.ipynb
# https://github.com/Open-EO/openeo-community-examples/tree/main/python

### Select Area of Interest

In [ ]:
center = [46.497012, 11.356429]
zoom = 16
eo_map = openeoMap(center, zoom)
eo_map.map

In [ ]:
bbox = eo_map.getBbox()
print('west',bbox[0],'\neast',bbox[2],'\nsouth',bbox[1],'\nnorth',bbox[3])

### Recreate process graph

In [ ]:
collection      = 'SENTINEL2_L2A_SENTINELHUB'
spatial_extent  = {'west':bbox[0],'east':bbox[2],'south':bbox[1],'north':bbox[3],'crs':4326}
temporal_extent = ["2018-02-01", "2018-02-15"]
bands           = ['B03', 'B11', 'CLM']

In [ ]:
s2cube = conn.load_collection(collection,
                          spatial_extent=spatial_extent,
                          bands=bands,
                          temporal_extent=temporal_extent)

#### Cloud Mask Extraction

cloud_mask= s2cube.band("CLM")
s2cube_masked = s2cube.mask(cloud_mask)

# cloud_mask = cloud_mask.reduce_dimension(dimension="t", reducer="mean")

#### NDSI Computation

In [ ]:
green = s2cube_masked.band("B03")
swir = s2cube_masked.band("B11")
ndsi = (green - swir) / (green + swir)
snowmap = ndsi > 0.4
# reduce along time
snowmap  = snowmap.reduce_dimension(dimension="t", reducer="mean")

In [ ]:
snowmap = snowmap.save_result(format="GTiff")
snowmap_job = snowmap.create_job(title="Snow map computation")
snowmap_job

In [ ]:
snowmap_job.start_and_wait()

In [ ]:
snowmap_results = snowmap_job.get_results()
snowmap_results.download_files("data/snowmap/")

### Data sharing with STAC
STAC: SpatioTemporal Assets Catalog


In [ ]:
import datetime

import pystac
from pystac.utils import str_to_datetime

import rasterio

# Import extension version
from rio_stac.stac import PROJECTION_EXT_VERSION, RASTER_EXT_VERSION, EO_EXT_VERSION

# Import rio_stac methods
from rio_stac.stac import (
    get_dataset_geom,
    get_projection_info,
    get_raster_info,
    get_eobands_info,
    bbox_to_geom,
)

In [ ]:
assets = [
    {"name": "Snowmap_1", "path": "data/snowmap/1/default.tif", "href": None,"role": None}
    {"name": "Snowmap_2", "path": "data/snowmap/2/default.tif", "href": None,"role": None}
]

media_type = pystac.MediaType.COG #rio_stac.stac.get_media_type
properties = {}
input_datetime = None
id = "snowmap_stac"
collection = "Snowmpa Bolzano"
collection_url = None

extensions =[
    f"https://stac-extensions.github.io/projection/{PROJECTION_EXT_VERSION}/schema.json", 
    f"https://stac-extensions.github.io/raster/{RASTER_EXT_VERSION}/schema.json",
    f"https://stac-extensions.github.io/eo/{EO_EXT_VERSION}/schema.json",
]


In [ ]:
bboxes = []

pystac_assets = []

img_datetimes = []

for asset in assets:
    with rasterio.open(asset["path"]) as src_dst:
        # Get BBOX and Footprint
        dataset_geom = get_dataset_geom(src_dst, densify_pts=0, precision=-1)
        bboxes.append(dataset_geom["bbox"])

        if "start_datetime" not in properties and "end_datetime" not in properties:
            # Try to get datetime from https://gdal.org/user/raster_data_model.html#imagery-domain-remote-sensing
            dst_date = src_dst.get_tag_item("ACQUISITIONDATETIME", "IMAGERY")
            dst_datetime = str_to_datetime(dst_date) if dst_date else None
            if dst_datetime:
                img_datetimes.append(dst_datetime)

        proj_info = {
            f"proj:{name}": value
            for name, value in get_projection_info(src_dst).items()
        }

        raster_info = {
            "raster:bands": get_raster_info(src_dst, max_size=1024)
        }

        eo_info = {}
        eo_info = {"eo:bands": get_eobands_info(src_dst)}
        cloudcover = src_dst.get_tag_item("CLOUDCOVER", "IMAGERY")
        if cloudcover is not None:
            properties.update({"eo:cloud_cover": int(cloudcover)})

        pystac_assets.append(
            (
                asset["name"], 
                pystac.Asset(
                    href=asset["href"] or src_dst.name,
                    media_type=media_type,
                    extra_fields={
                        **proj_info,
                        **raster_info, 
                        **eo_info
                    },
                    roles=asset["role"],
                ),
            )
        )

if img_datetimes and not input_datetime:
    input_datetime = img_datetimes[0]
    
input_datetime = input_datetime or datetime.datetime.utcnow()    

minx, miny, maxx, maxy = zip(*bboxes)
bbox = [min(minx), min(miny), max(maxx), max(maxy)]
            
# item
item = pystac.Item(
    id=id,
    geometry=bbox_to_geom(bbox),
    bbox=bbox,
    collection=collection,
    stac_extensions=extensions,
    datetime=input_datetime,
    properties=properties,
)

# if we add a collection we MUST add a link
if collection:
    item.add_link(
        pystac.Link(
            pystac.RelType.COLLECTION,
            collection_url or collection,
            media_type=pystac.MediaType.JSON,
        )
    )

for key, asset in pystac_assets:
    item.add_asset(key=key, asset=asset)

In [ ]:
item.validate()

In [ ]:
import json

print(json.dumps(item.to_dict(), indent=4))

#### Validate STAC

In [ ]:
#https://github.com/stac-utils/stac-validator
from stac_validator import stac_validator

stac = stac_validator.StacValidate()
stac.validate_dict(json.dumps(item.to_dict(), indent=4))
print(stac.message)